# Federated Learning Case Study

## 04. Federated Learning Example using CICIDS2017 (Improved dataset) with Flower and Tensorflow/Keras

In this notebook we use the Flower Federated Learning library (flower.dev) with Tensorflow/Keras to distribute the CICIDS2017 data across multiple clients.


### 1. Preparing the CICIDS dataset

In [2]:
import os
import pandas as pd
import numpy as np

# We load in the data generated from notebook two of this series
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [2]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(1679923, 79)
(419981, 79)
(1679923,)
(419981,)


### 2. Set up the Federated Learning system

In [4]:
from collections import OrderedDict
from typing import List, Tuple

In [5]:
NUM_OF_CLIENTS = 10
NUM_OF_ROUNDS = 5

In [58]:
# We use the sklearn function for Stratified KFold groups, to distribute a balanced set of examples for each client.
# Further experimentation will look at different splits of how the data is distributed amongst the clients.

import numpy as np

fl_X_train = []
fl_y_train = []

METHODS = ['stratified', 'split_by_attack', 'split_by_count'] 
METHOD = METHODS[0]


if METHOD == 'stratified':
    ## 1. STRATIFIED SAMPLING
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=NUM_OF_CLIENTS, shuffle=True, random_state=42)
    skf.get_n_splits(X_train, y_train)

    for train_index, test_index in skf.split(X_train, y_train):
        fl_X_train.append(X_train[test_index])
        fl_y_train.append(y_train[test_index]) 

elif METHOD == 'split_by_attack':
    for i in np.unique(y_test):
        print ("Get class: ", i)
        indices = np.where(y_train==i)
        print ("Shape of class ", i , " : ", X_train[indices].shape)
        fl_X_train.append(X_train[indices])
        fl_y_train.append(y_train[indices]) 

elif METHOD == 'split_by_count':
    COUNT = 10000
    s = np.arange(0,X_train.shape[0],COUNT)
    for i in range(len(s)-1):
        fl_X_train.append(X_train[s[i]:s[i+1]])
        fl_y_train.append(y_train[s[i]:s[i+1]])

NUM_OF_CLIENTS = len(fl_X_train)
print ("NUM_OF_CLIENTS:", NUM_OF_CLIENTS)

    


NUM_OF_CLIENTS: 167


In [60]:
fl_X_train[0].shape

(10000, 79)

In [61]:
for i in range(len(fl_X_train)):
    print (i, ':', "X shape", fl_X_train[0].shape, " Y shape:" , fl_y_train[0].shape)

0 : X shape (10000, 79)  Y shape: (10000,)
1 : X shape (10000, 79)  Y shape: (10000,)
2 : X shape (10000, 79)  Y shape: (10000,)
3 : X shape (10000, 79)  Y shape: (10000,)
4 : X shape (10000, 79)  Y shape: (10000,)
5 : X shape (10000, 79)  Y shape: (10000,)
6 : X shape (10000, 79)  Y shape: (10000,)
7 : X shape (10000, 79)  Y shape: (10000,)
8 : X shape (10000, 79)  Y shape: (10000,)
9 : X shape (10000, 79)  Y shape: (10000,)
10 : X shape (10000, 79)  Y shape: (10000,)
11 : X shape (10000, 79)  Y shape: (10000,)
12 : X shape (10000, 79)  Y shape: (10000,)
13 : X shape (10000, 79)  Y shape: (10000,)
14 : X shape (10000, 79)  Y shape: (10000,)
15 : X shape (10000, 79)  Y shape: (10000,)
16 : X shape (10000, 79)  Y shape: (10000,)
17 : X shape (10000, 79)  Y shape: (10000,)
18 : X shape (10000, 79)  Y shape: (10000,)
19 : X shape (10000, 79)  Y shape: (10000,)
20 : X shape (10000, 79)  Y shape: (10000,)
21 : X shape (10000, 79)  Y shape: (10000,)
22 : X shape (10000, 79)  Y shape: (10000,

In [9]:
# Checking the uniqueness of the class labels
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

In [10]:
%%capture
!pip install flwr[simulation] tensorflow

In [11]:
import flwr as fl
import numpy as np
import tensorflow as tf

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


flwr 1.0.0
numpy 1.22.4
tf 2.10.0


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

In [16]:
class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=1, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels)
        print ("Client ", self.cid, "Evaluating complete...", accuracy, loss)
        return loss, len(self.train_data), {"accuracy": accuracy}

In [17]:

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      Dense(18, activation='sigmoid'),  
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)

In [18]:
from flwr.common import Metrics
from sklearn.metrics import log_loss

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    accuracy = sum(accuracies) / sum(examples)
    print ("Accuracy:", accuracy)
    return {"accuracy": sum(accuracies) / sum(examples)}



def get_evaluate_fn(server_model):
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    def evaluate(server_round, parameters, config):
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print ("Server Evaluating...")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        print ("Server Evaluating complete...", accuracy, loss)
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      Dense(18, activation='sigmoid'),  
    ])
server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

INFO flower 2022-10-28 23:13:37,066 | app.py:142 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-28 23:13:47,807 | app.py:176 | Flower VCE: Ray initialized with resources: {'memory': 13725664052.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 6862832025.0, 'CPU': 8.0}
INFO flower 2022-10-28 23:13:47,810 | server.py:86 | Initializing global parameters
INFO flower 2022-10-28 23:13:47,811 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-10-28 23:13:58,211 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-10-28 23:13:58,212 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=19964) Client ID: 3
Server Evaluating...
13125/13125 [==============================] - 26s 2ms/step - loss: 2.8677 - accuracy: 0.0014


INFO flower 2022-10-28 23:14:25,438 | server.py:91 | initial parameters (loss, other metrics): 2.867741584777832, {'accuracy': 0.0014191118534654379}
INFO flower 2022-10-28 23:14:25,439 | server.py:101 | FL starting
DEBUG flower 2022-10-28 23:14:25,440 | server.py:215 | fit_round 1: strategy sampled 10 clients (out of 10)


Server Evaluating complete... 0.0014191118534654379 2.867741584777832
(launch_and_fit pid=19964) Client ID: 5
(launch_and_fit pid=19964) Client  5 Training...
(launch_and_fit pid=19964) 
(launch_and_fit pid=19964)    1/5250 [..............................] - ETA: 2:40:06 - loss: 2.8628 - accuracy: 0.0000e+00
(launch_and_fit pid=19964) 
(launch_and_fit pid=19964)   16/5250 [..............................] - ETA: 25s - loss: 1.3549 - accuracy: 0.6973        
(launch_and_fit pid=19964) 
(launch_and_fit pid=19964)   31/5250 [..............................] - ETA: 24s - loss: 1.1119 - accuracy: 0.7349
(launch_and_fit pid=19964)   46/5250 [..............................] - ETA: 23s - loss: 0.9687 - accuracy: 0.7541
(launch_and_fit pid=19964) 
(launch_and_fit pid=19964)   56/5250 [..............................] - ETA: 24s - loss: 0.8897 - accuracy: 0.7634
(launch_and_fit pid=19964) 
(launch_and_fit pid=19964)   64/5250 [..............................] - ETA: 25s - loss: 0.8481 - accuracy: 0.

DEBUG flower 2022-10-28 23:15:52,878 | server.py:229 | fit_round 1 received 10 results and 0 failures
WARNING flower 2022-10-28 23:15:52,916 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


...

INFO flower 2022-10-28 23:23:08,456 | server.py:116 | fit progress: (5, 0.020442605018615723, {'accuracy': 0.9924091696739197}, 523.0166051000001)
DEBUG flower 2022-10-28 23:23:08,458 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 10)


Server Evaluating complete... 0.9924091696739197 0.020442605018615723
(launch_and_evaluate pid=19964) Client ID: 5
(launch_and_evaluate pid=17588) Client ID: 0
(launch_and_evaluate pid=17588) Client  0 Evaluating...
(launch_and_evaluate pid=15344) Client ID: 1
(launch_and_evaluate pid=11688) Client ID: 3
(launch_and_evaluate pid=19964) Client  5 Evaluating...
(launch_and_evaluate pid=15344) Client  1 Evaluating...
(launch_and_evaluate pid=17588) Client ID: 8
(launch_and_evaluate pid=11688) Client  3 Evaluating...


DEBUG flower 2022-10-28 23:23:09,555 | server.py:179 | evaluate_round 5 received 0 results and 5 failures
INFO flower 2022-10-28 23:23:09,559 | server.py:144 | FL finished in 524.1180017000001
INFO flower 2022-10-28 23:23:09,560 | app.py:180 | app_fit: losses_distributed []
INFO flower 2022-10-28 23:23:09,563 | app.py:181 | app_fit: metrics_distributed {}
INFO flower 2022-10-28 23:23:09,565 | app.py:182 | app_fit: losses_centralized [(0, 2.867741584777832), (1, 0.04431663081049919), (2, 0.02608165331184864), (3, 0.022989749908447266), (4, 0.021301809698343277), (5, 0.020442605018615723)]
INFO flower 2022-10-28 23:23:09,566 | app.py:183 | app_fit: metrics_centralized {'accuracy': [(0, 0.0014191118534654379), (1, 0.9856279492378235), (2, 0.9910258054733276), (3, 0.992002010345459), (4, 0.9924830198287964), (5, 0.9924091696739197)]}


(launch_and_evaluate pid=17588) Client  8 Evaluating...


History (loss, centralized):
	round 0: 2.867741584777832
	round 1: 0.04431663081049919
	round 2: 0.02608165331184864
	round 3: 0.022989749908447266
	round 4: 0.021301809698343277
	round 5: 0.020442605018615723
History (metrics, centralized):
{'accuracy': [(0, 0.0014191118534654379), (1, 0.9856279492378235), (2, 0.9910258054733276), (3, 0.992002010345459), (4, 0.9924830198287964), (5, 0.9924091696739197)]}